In [1]:
import pandas as pd 
import numpy as np
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.utils import to_categorical
from keras.optimizers import RMSprop
from keras.layers import Dense, GlobalAveragePooling2D,Input,Flatten,Dropout
from keras import regularizers
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

Using TensorFlow backend.
C:\Users\lenovo\Anaconda3\envs\py35\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
path = "C:/Users/lenovo/Documents/CS/6Sem/CG/newtrain/"

In [3]:
x_train = np.load("C:/Users/lenovo/Documents/CS/6Sem/CG/Labelling-Paintings/Data/x_train.npy")
x_test = np.load("C:/Users/lenovo/Documents/CS/6Sem/CG/Labelling-Paintings/Data/x_test.npy")
y_train = np.load("C:/Users/lenovo/Documents/CS/6Sem/CG/Labelling-Paintings/Data/y_train.npy")
y_test = np.load("C:/Users/lenovo/Documents/CS/6Sem/CG/Labelling-Paintings/Data/y_test.npy")

In [4]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((13680,), (13680,), (13680,), (3420,))

In [5]:
le = preprocessing.LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train) 
y_test = le.transform(y_test)
no_classes = len(np.unique(y_train))
print("Number of classes: " + str(no_classes))
y_train = to_categorical(y_train, num_classes = no_classes)
y_test = to_categorical(y_test, num_classes = no_classes)

Number of classes: 57


In [6]:
inter_model = VGG16(weights='imagenet', include_top=False)

In [13]:
def generator1(data,y, im_height = 224, im_width = 224, train = True):
    while 1:
        for i in range(len(data)//5):
            labels = []
            features = []
            images = np.zeros((5,224,224,3))
            for idx, image_path in enumerate(data[5*i:5*(i+1)]):
                img = image.img_to_array(image.load_img(path + str(image_path)))
                images[idx] = img
                if(train):
                    labels.append(y[idx])

            images = preprocess_input(images)
            features = inter_model.predict(images)
            if(train):
                labels = np.array(labels)
                yield features, labels
            else:
                yield features

In [14]:
input_layer = Input(shape=(7,7,512))
f1=Flatten()(input_layer)
y = Dense(1024, activation='relu',kernel_initializer='glorot_normal',kernel_regularizer=regularizers.l2(0.05))(f1)
y=Dropout(0.5)(y)
y = Dense(1024, activation='relu',kernel_initializer='glorot_normal',kernel_regularizer=regularizers.l2(0.05))(y)
y=Dropout(0.5)(y)
#y = Dense(1024, activation='relu',kernel_initializer='glorot_normal',kernel_regularizer=regularizers.l2(0.01),activity_regularizer=regularizers.l1(0.01))(y)
#y=Dropout(0.5)(y)
predictions = Dense(no_classes, activation='softmax',kernel_initializer='glorot_normal')(y)
model = Model(inputs=input_layer, outputs=predictions)
rms=RMSprop(lr=0.0001)
model.compile(optimizer=rms, loss='categorical_crossentropy',metrics=['accuracy', 'top_k_categorical_accuracy'])

In [23]:
# fit_generator doesn't work if this isn't executed. No idea why
labels = []
images = np.zeros((32,224,224,3))
train = 1
i = 0
for idx, image_path in enumerate(x_train[32*i:32*(i+1)]):
    img = image.img_to_array(image.load_img(path + str(image_path)))
    images[idx] = img
    if(train):
        labels.append(y[idx])

images = preprocess_input(images)
features = inter_model.predict(images)

In [15]:
model.fit_generator(generator1(x_train,y_train),steps_per_epoch=x_train.shape[0]//5, epochs = 50, validation_data = generator1(x_test,y_test),validation_steps = y_test.shape[0]//5,callbacks= [EarlyStopping(patience=4)])

Epoch 1/50


Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\lenovo\Anaconda3\envs\py35\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Users\lenovo\Anaconda3\envs\py35\lib\threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\lenovo\Anaconda3\envs\py35\lib\site-packages\keras\engine\training.py", line 612, in data_generator_task
    generator_output = next(self._generator)
  File "<ipython-input-13-ab129c86ed28>", line 14, in generator1
    features = inter_model.predict(images)
  File "C:\Users\lenovo\Anaconda3\envs\py35\lib\site-packages\keras\engine\training.py", line 1582, in predict
    self._make_predict_function()
  File "C:\Users\lenovo\Anaconda3\envs\py35\lib\site-packages\keras\engine\training.py", line 1049, in _make_predict_function
    **kwargs)
  File "C:\Users\lenovo\Anaconda3\envs\py35\lib\site-packages\keras\backend\tensorflow_backend.py", line 2251, in function
    return Functio

ValueError: output of generator should be a tuple `(x, y, sample_weight)` or `(x, y)`. Found: None

In [16]:
# fit_generator doesn't work if this isn't executed. No idea why
labels = []
images = np.zeros((32,224,224,3))
train = 1
i = 0
for idx, image_path in enumerate(x_train[32*i:32*(i+1)]):
    img = image.img_to_array(image.load_img(path + str(image_path)))
    images[idx] = img
    if(train):
        labels.append(y[idx])

images = preprocess_input(images)
features = inter_model.predict(images)

In [17]:
features.shape

(32, 7, 7, 512)